<a href="https://colab.research.google.com/github/wangbxj1234/5001/blob/main/cos_as_pe_flow_as_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn.functional as F
import numpy as np

from torch import Tensor
from typing import Optional
from torch import nn

o=[ 838, 2088, 2317, 3567, 4541, 5561, 6427, 7077]
o=torch.IntTensor(o)
x = torch.rand(7077, 128)
p = torch.rand(7077, 3)

In [3]:
class CosformerAttention_pe(nn.Module):
    """
    cosformer attention in "cosFormer: Rethinking Softmax In Attention"
    https://arxiv.org/abs/2202.08791
    """
    def __init__(self,in_planes):
        super().__init__()

        self.act_fun = self.get_act_fun()
        # q, k, v projection
        self.k_proj = nn.Linear(in_planes, in_planes)
        self.v_proj = nn.Linear(in_planes, in_planes)
        self.q_proj = nn.Linear(in_planes, in_planes)
        #self.p_proj = nn.Linear(3, in_planes)
        self.p_proj_q = nn.Sequential(nn.Linear(3, 3), nn.BatchNorm1d(3), nn.ReLU(inplace=True), nn.Linear(3, in_planes))
        self.p_proj_k = nn.Sequential(nn.Linear(3, 3), nn.BatchNorm1d(3), nn.ReLU(inplace=True), nn.Linear(3, in_planes))
        # outprojection
        self.out_proj = nn.Linear(in_planes, 3)

    def get_index(self, seq_len):
        index = np.pi / 2 * torch.arange(1, seq_len + 1).reshape(1, -1, 1)
        return nn.Parameter(index, requires_grad=False)

    def get_act_fun(self):
        return F.relu
    def forward(self, pxo) -> torch.Tensor:
        p, x, o = pxo 
        eps = 1e-6
     
        n=0;
        x_sub=[]
        p_sub=[]
        out=[]
        for i in o:
           x_s=x[n:int(i),:]
           x_sub.append(x_s)
           p_s=p[n:int(i),:]
           p_sub.append(p_s)
           tgt_len, embed_dim = x_s.size() ###(N,d) 

           n=int(i)

           q = self.q_proj(x_s)######
        # (N,d)
           k = self.k_proj(x_s)########
        # (N,d)
           v = self.v_proj(x_s)######
           
           pe_q = self.p_proj_q(p_s)
           pe_k = self.p_proj_k(p_s)

    
           q = q+pe_q
           k = k+pe_k
        # activation
           q = self.act_fun(q)####Q'
           k = self.act_fun(k)####K'
        
        # cos transform
           m = tgt_len
        # get index and send to cuda
           weight_index = self.get_index(m).to(q)########(pi*[1,m,1])/2

        # (N, 2 * d)
           q_ = torch.cat([q * torch.sin(weight_index[:tgt_len, :] / m), q * torch.cos(weight_index[ :tgt_len, :] / m)], dim=-1).squeeze(0)
        # (N, 2 * d)
           k_ = torch.cat([k * torch.sin(weight_index[:tgt_len, :] / m), k * torch.cos(weight_index[ :tgt_len, :] / m)], dim=-1).squeeze(0)

        
            # ( L, 2 * d) ( L, d) -> ( 2 * d, d)
           kv_ = torch.einsum('ld,lm->dm', k_, v)
            # ( L, 2 * d) ( 2 * d) -> ( L)
           z_ = 1 / torch.clamp_min(torch.einsum('ld,d->l', q_, torch.sum(k_, axis=0)), eps)
            # ( L, 2 * d) ( d, 2 * d) ( L) -> ( L, d)
           attn_output = self.out_proj(torch.einsum('ld,dm,l->lm', q_, kv_, z_))
           out.append(attn_output)
        #out=torch.IntTensor(out)
        out=torch.cat(out,dim=0)

        return out
class Flow_Attention(nn.Module):
    # flow attention in normal version
    def __init__(self, in_planes):
        super(Flow_Attention, self).__init__()
        self.n_heads =32
        self.query_projection = nn.Linear(in_planes, in_planes)
        self.key_projection = nn.Linear(in_planes, in_planes)
        ####加入位置编码
        self.p_proj_q = nn.Sequential(nn.Linear(3, 3), nn.BatchNorm1d(3), nn.ReLU(inplace=True), nn.Linear(3, in_planes))
        self.p_proj_k = nn.Sequential(nn.Linear(3, 3), nn.BatchNorm1d(3), nn.ReLU(inplace=True), nn.Linear(3, in_planes))
        ####
        self.value_projection = nn.Linear(in_planes, in_planes)
        self.out_projection = nn.Linear(in_planes, in_planes)
        self.dropout = nn.Dropout(0.05)
        self.eps = 1e-6

    def kernel_method(self, x):
        return torch.sigmoid(x)

    def dot_product(self, q, k, v):
        kv = torch.einsum("hld,hlm->hdm", k, v)
        qkv = torch.einsum("hld,hdm->hlm", q, kv)
        return qkv

    def forward(self, pxo) -> torch.Tensor:
        p, x, o = pxo   
        PN=0;
        x_sub=[]
        p_sub=[]
        out=[]
        for offset in o:
           x_s=x[PN:int(offset),:]
           x_sub.append(x_s)
           p_s=p[PN:int(offset),:]
           p_sub.append(p_s)
           tgt_len, embed_dim = x_s.size() ###(N,d) 
           #print(offset,tgt_len, embed_dim)
           PN=int(offset)
        ## input: (L or S) D; output: L D
        ## Note: queries, keys, values are not projected yet
        ## 1. Linear projection
           L=tgt_len
           S=tgt_len
           queries = self.query_projection(x_s).view(L, self.n_heads, -1)
           keys = self.key_projection(x_s).view(S, self.n_heads, -1)
           ####加入位置编码
           pe_q = self.p_proj_q(p_s).view(L, self.n_heads, -1)
           pe_k = self.p_proj_k(p_s).view(S, self.n_heads, -1)
           queries = queries+pe_q
           keys = keys+pe_k
           ####
           values = self.value_projection(x_s).view(S, self.n_heads, -1)
           queries = queries.transpose(0, 1)
           keys = keys.transpose(0, 1)
           values = values.transpose(0, 1)
           # 2. Non-negative projection
           queries = self.kernel_method(queries)
           keys = self.kernel_method(keys)
           ## 3. Flow-Attention
           # (1) Calculate incoming and outgoing flow
           sink_incoming = 1.0 / (torch.einsum("hld,hd->hl", queries + self.eps, keys.sum(dim=1) + self.eps))
           source_outgoing = 1.0 / (torch.einsum("hld,hd->hl", keys + self.eps, queries.sum(dim=1) + self.eps))
           # (2) conservation refine for source and sink
           conserved_sink = torch.einsum("hld,hd->hl", queries + self.eps,
                                      (keys * source_outgoing[:, :, None]).sum(dim=1) + self.eps)
           conserved_source = torch.einsum("hld,hd->hl", keys + self.eps,
                                        (queries * sink_incoming[:, :, None]).sum(dim=1) + self.eps)
           conserved_source = torch.clamp(conserved_source, min=-1.0, max=1.0)  # for stability
           # (3) Competition & Allocation
           sink_allocation = torch.sigmoid(conserved_sink * (float(queries.shape[1]) / float(keys.shape[1])))
           source_competition = torch.softmax(conserved_source, dim=-1) * float(keys.shape[1])
           # (4) dot product
           output = (self.dot_product(queries * sink_incoming[:, :, None],  # for value normalization
                              keys,
                              values * source_competition[:, :, None])  # competition
                * sink_allocation[:, :, None]).transpose(0, 1)  # allocation
           ## (5) Final projection
           output = output.reshape(L, -1)
           output = self.out_projection(output)
           output = self.dropout(output)

           out.append(output)
        #out=torch.IntTensor(out)
        out=torch.cat(out,dim=0)

        return out

In [5]:
planes=128
transformer1 = CosformerAttention_pe(planes)
transformer2 = Flow_Attention(planes)
p_new = transformer1([p, x, o])
right_res = transformer2([p_new, x, o])
print(right_res.shape)

torch.Size([7077, 128])
